In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets 
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable

In [2]:
def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

In [3]:
# Image processing 
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5, 0.5, 0.5), 
                                     std=(0.5, 0.5, 0.5))])
# MNIST dataset
mnist = datasets.MNIST(root='./data/',
                       train=True,
                       transform=transform,
                       download=True)

In [4]:
# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=100, 
                                          shuffle=True)

In [5]:
# Discriminator
D = nn.Sequential(
    nn.Linear(784, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 1),
    nn.Sigmoid())

# Generator 
G = nn.Sequential(
    nn.Linear(64, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 784),
    nn.Tanh())

if torch.cuda.is_available():
    D.cuda()
    G.cuda()

In [6]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0003)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0003)

In [7]:
# Start training
for epoch in range(200):
    for i, (images, _) in enumerate(data_loader):
        # Build mini-batch dataset
        batch_size = images.size(0)
        images = to_var(images.view(batch_size, -1))
        real_labels = to_var(torch.ones(batch_size))
        fake_labels = to_var(torch.zeros(batch_size))

        #============= Train the discriminator =============#
        # Compute loss with real images
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute loss with fake images
        z = to_var(torch.randn(batch_size, 64))
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop + Optimize
        d_loss = d_loss_real + d_loss_fake
        D.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        #=============== Train the generator ===============#
        # Compute loss with fake images
        z = to_var(torch.randn(batch_size, 64))
        fake_images = G(z)
        outputs = D(fake_images)
        g_loss = criterion(outputs, real_labels)
        
        # Backprop + Optimize
        D.zero_grad()
        G.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 300 == 0:
            print('Epoch [%d/%d], Step[%d/%d], d_loss: %.4f, '
                  'g_loss: %.4f, D(x): %.2f, D(G(z)): %.2f' 
                  %(epoch, 200, i+1, 600, d_loss.data[0], g_loss.data[0],
                    real_score.data.mean(), fake_score.data.mean()))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.view(images.size(0), 1, 28, 28)
        save_image(denorm(images.data), './data/real_images.png')
    
    # Save sampled images
    fake_images = fake_images.view(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images.data), './data/fake_images-%d.png' %(epoch+1))

# Save the trained parameters 
torch.save(G.state_dict(), './generator.pkl')
torch.save(D.state_dict(), './discriminator.pkl')

Epoch [0/200], Step[300/600], d_loss: 0.2668, g_loss: 5.8942, D(x): 0.94, D(G(z)): 0.09
Epoch [0/200], Step[600/600], d_loss: 0.0904, g_loss: 5.0523, D(x): 0.95, D(G(z)): 0.02
Epoch [1/200], Step[300/600], d_loss: 0.6151, g_loss: 4.1212, D(x): 0.87, D(G(z)): 0.27
Epoch [1/200], Step[600/600], d_loss: 0.6187, g_loss: 3.2803, D(x): 0.82, D(G(z)): 0.19
Epoch [2/200], Step[300/600], d_loss: 0.7462, g_loss: 1.9519, D(x): 0.75, D(G(z)): 0.30
Epoch [2/200], Step[600/600], d_loss: 0.5329, g_loss: 2.5638, D(x): 0.82, D(G(z)): 0.22
Epoch [3/200], Step[300/600], d_loss: 0.7772, g_loss: 1.7979, D(x): 0.73, D(G(z)): 0.21
Epoch [3/200], Step[600/600], d_loss: 0.4610, g_loss: 2.0244, D(x): 0.84, D(G(z)): 0.18
Epoch [4/200], Step[300/600], d_loss: 1.3985, g_loss: 1.1188, D(x): 0.62, D(G(z)): 0.42
Epoch [4/200], Step[600/600], d_loss: 1.4541, g_loss: 2.3461, D(x): 0.72, D(G(z)): 0.47
Epoch [5/200], Step[300/600], d_loss: 0.2211, g_loss: 3.7069, D(x): 0.92, D(G(z)): 0.10
Epoch [5/200], Step[600/600], d_

KeyboardInterrupt: 